In [1]:
import mlflow

In [2]:
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='/Users/gbemidebe/Documents/GitHub/mlops-zoomcamp/02-experiment-tracking/examples/case02/mlruns/1', creation_time=1716753193765, experiment_id='1', last_update_time=1716753193765, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [3]:
# get data: check if directory exist. if yes, skip else create new one
import os, wget
data_directory = 'data/'
if os.path.exists(data_directory):
    print("Directory exists")
else:
    os.mkdir(data_directory)
    print("Directory does not exist and created new one")

# set directory
os.chdir(f'./{data_directory}')
# Download files
remote_desktop = False
if remote_desktop:
    if not os.path.exists('green_tripdata_2021-01.parquet'):
        !wget https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet
    if not os.path.exists('green_tripdata_2021-02.parquet'):
        !wget https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet
else:
    if not os.path.exists('green_tripdata_2021-01.parquet'):
        wget.download('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet')
    if not os.path.exists('green_tripdata_2021-02.parquet'):
        wget.download('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet')
os.chdir(f'../')

Directory exists


In [4]:
import pandas as pd
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [5]:
df_train = read_dataframe('./data/green_tripdata_2021-01.parquet').iloc[:2000]
df_val = read_dataframe('./data/green_tripdata_2021-02.parquet').iloc[:2000]

In [6]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [7]:
from sklearn.feature_extraction import DictVectorizer

categorical = ['PU_DO'] 
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [8]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [9]:
# save the dictionary vectorizer
import pickle
if os.path.exists("saved_models"):
    print("Directory exists")
else:
    os.mkdir("saved_models")
    print("Directory does not exist and created new one")
    
with open("saved_models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)

Directory exists


In [10]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR
from sklearn.metrics import root_mean_squared_error, r2_score

mlflow.sklearn.autolog()

for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):

    with mlflow.start_run():

        mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
        mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")
        mlflow.log_artifact("saved_models/preprocessor.b", artifact_path="preprocessor")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val)
        rmse = root_mean_squared_error(y_val, y_pred)
        r2 = r2_score(y_val, y_pred)

        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("r2", r2)
        

2024/05/26 15:35:08 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
2024/05/26 15:35:10 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/gbemidebe/miniconda3/envs/exp-tracking-env/lib/python3.9/site-packages/_distutils_hack/__init__.py:26: UserWarning: Setuptools is replacing distutils."
2024/05/26 15:35:10 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
2024/05/26 15:35:12 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
2024/05/26 15:35:14 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
/Users/gbemidebe/miniconda3/envs/exp-tracking-env/lib/python3.9/site-packages/

In [11]:
import xgboost as xgb

with mlflow.start_run():
    
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)
    
    # culled from the instructor solution
    best_params = {
        'learning_rate': 0.09585355369315604,
        'max_depth': 30,
        'min_child_weight': 1.060597050922164,
        'objective': 'reg:linear',
        'reg_alpha': 0.018060244040060163,
        'reg_lambda': 0.011658731377413597,
        'seed': 42
    }

    mlflow.xgboost.autolog()

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50, verbose_eval=False
    )

    y_pred = booster.predict(valid)
    rmse = root_mean_squared_error(y_val, y_pred)

    with open("saved_models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)



/Users/gbemidebe/miniconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [15:35:15] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)
2024/05/26 15:35:17 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.
2024/05/26 15:35:17 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/gbemidebe/miniconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [15:35:17] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified."
